In [1]:
import wandb
import weave
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vincenttu. Use `wandb login --relogin` to force relogin


True

In [2]:
from agential.prompting.standard.prompting import Standard
import os

from agential.prompting.standard.prompts import (
    STANDARD_INSTRUCTION_AMBIGNQ, 
    STANDARD_INSTRUCTION_FEVER, 
    STANDARD_INSTRUCTION_GSM8K,  
    STANDARD_INSTRUCTION_HOTPOTQA, 
    STANDARD_INSTRUCTION_SVAMP, 
    STANDARD_INSTRUCTION_TRIVIAQA,
    STANDARD_INSTRUCTION_TABMWP,
    STANDARD_INSTRUCTION_HUMANEVAL,
    STANDARD_INSTRUCTION_MBPP,
)
from agential.core.fewshots.ambignq import AMBIGNQ_FEWSHOT_EXAMPLES_DIRECT
from agential.core.fewshots.fever import FEVER_FEWSHOT_EXAMPLES_DIRECT
from agential.core.fewshots.gsm8k import GSM8K_FEWSHOT_EXAMPLES_POT
from agential.core.fewshots.hotpotqa import HOTPOTQA_FEWSHOT_EXAMPLES_DIRECT
from agential.core.fewshots.svamp import SVAMP_FEWSHOT_EXAMPLES_POT
from agential.core.fewshots.triviaqa import TRIVIAQA_FEWSHOT_EXAMPLES_DIRECT
from agential.core.fewshots.tabmwp import TABMWP_FEWSHOT_EXAMPLES_POT
from agential.core.fewshots.humaneval import HUMANEVAL_FEWSHOT_EXAMPLES_POT
from agential.core.fewshots.mbpp import MBPP_FEWSHOT_EXAMPLES_POT

import warnings
warnings.filterwarnings('ignore')

from dotenv import load_dotenv
load_dotenv()

from agential.core.llm import LLM

llm = LLM("gpt-3.5-turbo", organization=os.getenv("OPENAI_ORGANIZATION"))

In [3]:

import json

# Open and read the JSON file
with open('../data/hotpot_dev_v1_simplified.json', 'r') as file:
    data = json.load(file)

# Print the data
print(data[:5])

[{'question': 'Were Scott Derrickson and Ed Wood of the same nationality?', 'answer': 'yes', 'type': 'comparison'}, {'question': 'What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?', 'answer': 'Chief of Protocol', 'type': 'bridge'}, {'question': 'What science fantasy young adult series, told in first person, has a set of companion books narrating the stories of enslaved worlds and alien species?', 'answer': 'Animorphs', 'type': 'bridge'}, {'question': 'Are the Laleli Mosque and Esma Sultan Mansion located in the same neighborhood?', 'answer': 'no', 'type': 'comparison'}, {'question': 'The director of the romantic comedy "Big Stone Gap" is based in what New York city?', 'answer': 'Greenwich Village, New York City', 'type': 'bridge'}]


In [8]:
from agential.eval.metrics.classification import EM, f1, precision, recall

# Log numbers (EM, f1, precission, recall)
# Log/Save artifacts to your runs
# Optional, how to log final metrics 

method = Standard(
        llm=llm,
        benchmark="hotpotqa",
    )

num_correct = 0 
samples = 20

wandb.init("standard_hotpotqa")
weave.init("standard_hotpotqa")

for i in data[:samples]:

    question = i["question"]
    answer = i["answer"]

    out = method.generate(
        question=question,
        key=answer
    )
    is_correct = EM(out.answer, answer)
    precision_out = precision(out.answer, answer)
    recall_out = recall(out.answer, answer)
    f1_out = f1(out.answer, answer)

    num_correct += int(is_correct)

wandb.finish()

ValueError: Project Mismatch: weave and wandb must be initialized using the same project. Found wandb.init targeting project "vincenttu/agential" and weave.init targeting project "vincenttu/standard_hotpotqa". To fix, please use the same project for both library initializations.

In [ ]:
from agential.eval.metrics.classification import normalize_answer


key = "Robert Erskine Childers DSC" 		 
answer = "Robert Erskine Childers"
key in answer or answer in key
# a = normalize_answer(answer)
# b =  normalize_answer(key)

EM(answer, key)